<a href="https://www.kaggle.com/code/dasaic98/world-income-sql-analysis?scriptVersionId=136563612" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Setup

In [1]:
import numpy as np 
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.preprocessing import MinMaxScaler
import warnings

# Filter or ignore specific warning messages
warnings.filterwarnings("ignore")
import sqlite3

database="../input/world-development-indicators/indicators.sqlite"
conn = sqlite3.connect(database)

tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", conn)
tables
# https://data.worldbank.org/indicator

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


,type,name,tbl_name,rootpage,sql
0,table,Country,Country,2,"CREATE TABLE Country (\n CountryCode TEXT,\..."
1,table,CountryNotes,CountryNotes,186,CREATE TABLE CountryNotes (\n Countrycode T...
2,table,Series,Series,948,"CREATE TABLE Series (\n SeriesCode TEXT,\n ..."
3,table,Indicators,Indicators,4448,CREATE TABLE Indicators (\n CountryName TEX...
4,table,SeriesNotes,SeriesNotes,1317550,CREATE TABLE SeriesNotes (\n Seriescode TEX...
5,table,Footnotes,Footnotes,1317587,CREATE TABLE Footnotes (\n Countrycode TEXT...


# Data Exploration

In [2]:
table = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name", conn)
print(table)

           name
0       Country
1  CountryNotes
2     Footnotes
3    Indicators
4        Series
5   SeriesNotes


In [3]:
query = 'SELECT * FROM indicators;'
indicators = pd.read_sql_query(query, conn)
fields=indicators.columns.to_list()
print(fields)

['CountryName', 'CountryCode', 'IndicatorName', 'IndicatorCode', 'Year', 'Value']


In [4]:
query="SELECT DISTINCT IndicatorName, IndicatorCode FROM Indicators WHERE IndicatorName LIKE '%National Income%';"
names=pd.read_sql_query(query, conn)
print("\nIncome related\n",names)
query="SELECT DISTINCT IndicatorName, IndicatorCode FROM Indicators WHERE IndicatorName LIKE '%Income Share%';"
names=pd.read_sql_query(query, conn)
print("\nIncome Share related\n",names)
query="SELECT DISTINCT IndicatorName, IndicatorCode FROM Indicators WHERE IndicatorName LIKE '%Poverty%';"
names=pd.read_sql_query(query, conn)
print("\nPoverty related\n",names)


Income related
                                        IndicatorName         IndicatorCode
0     Adjusted net national income (annual % growth)     NY.ADJ.NNTY.KD.ZG
1   Adjusted net national income (constant 2005 US$)        NY.ADJ.NNTY.KD
2         Adjusted net national income (current US$)        NY.ADJ.NNTY.CD
3  Adjusted net national income per capita (annua...  NY.ADJ.NNTY.PC.KD.ZG
4  Adjusted net national income per capita (const...     NY.ADJ.NNTY.PC.KD
5  Adjusted net national income per capita (curre...     NY.ADJ.NNTY.PC.CD

Income Share related
                       IndicatorName   IndicatorCode
0   Income share held by fourth 20%  SI.DST.04TH.20
1  Income share held by highest 10%  SI.DST.10TH.10
2  Income share held by highest 20%  SI.DST.05TH.20
3   Income share held by lowest 10%  SI.DST.FRST.10
4   Income share held by lowest 20%  SI.DST.FRST.20
5   Income share held by second 20%  SI.DST.02ND.20
6    Income share held by third 20%  SI.DST.03RD.20

Poverty related
  

# Data Analysis

In [5]:
indicator_codes=["NY.ADJ.NNTY.KD.ZG", 
                 "SI.DST.10TH.10", "SI.DST.05TH.20", "SI.DST.FRST.10","SI.DST.FRST.20",
                 "SI.POV.NAGP",
                 "SI.POV.NAHC","SI.POV.RUGP","SI.POV.RUHC","SI.POV.URGP","SI.POV.URHC"]

countries=["United States","Australia","United Kingdom","Canada","Germany","France",
           "Russian Federation", "China","Japan","South Korea","New Zealand"]

query="""SELECT CountryName, CountryCode, IndicatorCode, year, value
FROM indicators
WHERE year BETWEEN 2000 AND 2015
  AND countryname IN ("United States","Australia","United Kingdom","Canada","Germany","France","Russian Federation", "China","Japan","South Korea","New Zealand")
  AND indicatorcode IN ("NY.ADJ.NNTY.KD.ZG","SI.DST.FRST.20", "SI.DST.10TH.10", "SI.DST.05TH.20", "SI.DST.FRST.10","SI.DST.FRST.20","SI.POV.NAGP","SI.POV.NAHC","SI.POV.RUGP","SI.POV.RUHC","SI.POV.URGP","SI.POV.URHC");
"""

country_data=pd.read_sql_query(query, conn)
country_data

,CountryName,CountryCode,IndicatorCode,Year,Value
0,Australia,AUS,NY.ADJ.NNTY.KD.ZG,2000,2.456387
1,Australia,AUS,NY.ADJ.NNTY.KD.ZG,2001,3.017479
2,Australia,AUS,SI.DST.10TH.10,2001,25.240000
3,Australia,AUS,SI.DST.05TH.20,2001,41.150000
4,Australia,AUS,SI.DST.FRST.10,2001,2.310000
...,...,...,...,...,...
378,United States,USA,NY.ADJ.NNTY.KD.ZG,2013,2.394261
379,United States,USA,SI.DST.10TH.10,2013,30.190000
380,United States,USA,SI.DST.05TH.20,2013,46.440000
381,United States,USA,SI.DST.FRST.10,2013,1.700000


In [6]:
df=country_data[country_data["IndicatorCode"]=="NY.ADJ.NNTY.KD.ZG"]

fig=go.Figure()
fig.add_trace(go.Scatter(
            x=df["CountryName"], 
            y=df["Value"],
            mode='markers',
            showlegend=False,
            marker=dict(
                opacity=0.5,
                showscale=True,
            ))
        )

fig.update_layout(title_text='Adjusted net national income (annual % growth)')
fig.show()

In [7]:
df=country_data[country_data["IndicatorCode"]=="SI.DST.FRST.10"]

fig=go.Figure()
fig.add_trace(go.Scatter(
            x=df["CountryName"], 
            y=df["Value"],
            mode='markers',
            showlegend=False,
            marker=dict(
                opacity=0.5,
                showscale=True,
            ))
        )

fig.update_layout(title_text='Income share held by lowest 10%')
fig.show()

In [8]:
df=country_data[country_data["IndicatorCode"]=="SI.DST.FRST.20"]

fig=go.Figure()
fig.add_trace(go.Scatter(
            x=df["CountryName"], 
            y=df["Value"],
            mode='markers',
            showlegend=False,
            marker=dict(
                opacity=0.5,
                showscale=True,
            ))
        )

fig.update_layout(title_text='Income share held by lowest 20%')
fig.show()

In [9]:
df=country_data[country_data["IndicatorCode"]=="SI.DST.10TH.10"]

fig=go.Figure()
fig.add_trace(go.Scatter(
            x=df["CountryName"], 
            y=df["Value"],
            mode='markers',
            showlegend=False,
            marker=dict(
                opacity=0.5,
                showscale=True,
            ))
        )

fig.update_layout(title_text='Income share held by highest 10%')
fig.show()

In [10]:
df=country_data[country_data["IndicatorCode"]=="SI.DST.05TH.20"]

fig=go.Figure()
fig.add_trace(go.Scatter(
            x=df["CountryName"], 
            y=df["Value"],
            mode='markers',
            showlegend=False,
            marker=dict(
                opacity=0.5,
                showscale=True,
            ))
        )

fig.update_layout(title_text='Income share held by highest 20%')
fig.show() 